In [1]:
# 加载包
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

# 加载数据将每一个数据变形成32x32
train_data = pd.read_csv(r'D:/硕士相关/福瑞/train.csv', header=0)
test_data = pd.read_csv(r'D:/硕士相关/福瑞/test.csv', header=0)

# 对label进行映射替换
keys = list(np.unique(train_data[['Secondary folder']]))
values = list(range(len(keys)))
dic = dict(zip(keys, values))
print(dic)

# 进行label替换
train_data['Secondary folder'] = train_data.iloc[:, 3:4].replace(dic)
test_data['Secondary folder'] = test_data.iloc[:, 3:4].replace(dic)

X_train = np.array(train_data.iloc[:, -1024:])
y_train = np.array(train_data['Secondary folder']).astype(int)
X_test = np.array(test_data.iloc[:, -1024:])
y_test = np.array(test_data['Secondary folder']).astype(int)


X_test = torch.from_numpy(X_test).reshape(len(X_test), 1, 32, 32)
y_test = torch.from_numpy(y_test).squeeze()
X_train = torch.from_numpy(X_train).reshape(len(X_train), 1, 32, 32)
y_train = torch.from_numpy(y_train).squeeze()

print('X_test: ', X_test.shape,
      '\ny_test: ', y_test.shape,
      '\nX_train: ', X_train.shape,
      '\ny_train: ', y_train.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'D:/硕士相关/福瑞/train.csv'

In [4]:
############# 训练函数 #############
def train(net, train_features, train_labels, val_features, val_labels, num_epochs, 
          batch_size, optimizer, loss_function, def_init_weight=None, device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")): 
    """PARAMS:
    net: 需要进行训练的网络结构
    train_features: 训练features数据
    train_labels: 训练targets数据
    val_features: 验证features数据
    val_labels: 验证targets数据
    num_epochs: 需要训练的轮次
    batch_size: batch_size
    optimizer: 定义的优化器(torch.optim)
    loss_function: 自定义的损失函数(torch.nn)
    def_init_weight: 模型参数初始化(默认为None, torch.nn.init)
                        def init_weight(m):
                            if type(m) == nn.Linear or type(m) == nn.Conv2d:
                            nn.init.xavier_uniform_(m.weight)
    device: 自动搜索设备进行运行
                        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    """
    
    if def_init_weight != None:
        net.apply(def_init_weight)
        # net = torch.load('./Net_model.pt') # 使用预训练的pt文件
   
        
    dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    train_iter = DataLoader(dataset, batch_size, shuffle=True) 
    
    net = net.to(device)
    net.train()
    
    for epoch in range(num_epochs):
        loss_a=[]
        
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            output  = net(X.to(torch.float32))
            loss = loss_function(output, y.long())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_a.append(loss)

        if epoch % 30 == 0:
            print('loss_train: ', sum(loss_a)/len(train_iter))


## 进行训练

In [9]:
def init_weight(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
            
class Reshape(nn.Module):
    def forward(self, x):
        return x.view(-1, 1, 32, 32)
Net_model = nn.Sequential(
    Reshape(),
    nn.Conv2d(1, 64, 3, padding=1), 
    nn.ReLU(inplace=True),
    
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(64, 128, 3, padding=1), 
    nn.ReLU(inplace=True),
    
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Conv2d(128, 256, 3, padding=1), 
    nn.ReLU(inplace=True),
    
    nn.MaxPool2d(kernel_size=2, stride=2),
    
    nn.Flatten(),
    nn.Linear(256 * 4 * 4, 1024), 
    nn.ReLU(inplace=True),
    nn.Linear(1024, 512), 
    nn.ReLU(inplace=True),
    nn.Linear(512, 108))

num_epochs = 200
batch_size = 128
optimizer = torch.optim.SGD(Net_model.parameters(), lr = 0.5)
loss_function = nn.CrossEntropyLoss()

train(net=Net_model, train_features=X_train, train_labels=y_train, val_features=X_test, val_labels=y_test, num_epochs=num_epochs, 
          batch_size=batch_size, optimizer=optimizer, loss_function=loss_function, def_init_weight=init_weight, device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

torch.save(Net_model, 'Net_model.pt')

loss_train:  tensor(4.6975, device='cuda:0', grad_fn=<DivBackward0>)
loss_train:  tensor(4.6827, device='cuda:0', grad_fn=<DivBackward0>)
loss_train:  tensor(4.6827, device='cuda:0', grad_fn=<DivBackward0>)
loss_train:  tensor(4.6827, device='cuda:0', grad_fn=<DivBackward0>)
loss_train:  tensor(4.6826, device='cuda:0', grad_fn=<DivBackward0>)
loss_train:  tensor(4.6825, device='cuda:0', grad_fn=<DivBackward0>)
loss_train:  tensor(4.6826, device='cuda:0', grad_fn=<DivBackward0>)


## 使用test数据进行测试

In [10]:
net_test = torch.load('./Net_model.pt').to(device=torch.device('cpu'))
def evalution(X, y):
      test_pred = net_test(X.to(torch.float32))
      result = torch.max(test_pred, 1)[1].view(y.size())
      corrects = (result.data == y.data).sum().item()
      accuracy = corrects*100.0/len(X)
      loss_func = nn.CrossEntropyLoss()
      loss = loss_func(test_pred, y.long()).data.numpy()

      print(' Cross Entropy Loss: ', round(float(loss), 4), '\n',
            'Accuracy: ', accuracy, '%')
      
evalution(X_train, y_train)
evalution(X_test, y_test)

 Cross Entropy Loss:  4.6809 
 Accuracy:  1.1107300213703402 %
 Cross Entropy Loss:  4.6812 
 Accuracy:  1.1121603189591858 %
